In [1]:

import numpy as np
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Lambda, MaxPooling2D,
Flatten,ZeroPadding2D, BatchNormalization, Dense,Dropout,Activation,AveragePooling2D
from keras.layers import Input
from keras import regularizers
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


SyntaxError: trailing comma not allowed without surrounding parentheses (806870960.py, line 7)

### Load scrapped moves and grids

In [ ]:
moves = pd.read_csv('/kaggle/input/tetris/finalmoves.csv')

In [ ]:
grids = np.loadtxt('/kaggle/input/tetris/finalgrid.csv', delimiter=',')

In [ ]:
# reshape grids into (20,10)
grids = grids.reshape((int(grids.shape[0]/20),20,10))

### Create additional variable, height of each column of tetris grid

In [ ]:
def get_height(grid):
        locked = np.zeros(10)
        for i, column in enumerate(grid.T):
            j = 19
            height = 0
            while column[j] and j >= 0:
                height += 1
                j -= 1
            if locked[i] != height:
                locked[i] = height
        return locked

In [ ]:
heights = np.zeros((grids.shape[0],10))
for i, grid in enumerate(grids):
    heights[i] = get_height(grid)

#### Check if shape is the same

In [ ]:
assert grids.shape[0]== moves.shape[0]
assert grids.shape[0]== heights.shape[0]

In [ ]:
with open("heights.csv", "a") as outfile:
    np.savetxt(outfile, heights, fmt="%i", delimiter=",")

##### Analyze Height distribution

In [ ]:
# heights dist
def height_dist(heights):
    fig, axs = plt.subplots(2, 5,figsize=(15,17))
    
    i = 0
    for col in range(5):
        for row in range(2):
            axs[row,col].hist(heights.T[i])
            axs[row, col].set_title(f'Column {i+1}')
            
            i+=1
    plt.suptitle('Height distribution')
    fig.savefig('heights.png')


##### Analyze Move distribution

In [ ]:
def pie(moves):
    plt.pie(moves['move'].value_counts(),autopct='%1.1f%%')
    plt.title("Moves", loc = 'center')
    plt.legend(moves['move'].unique(),loc='upper right')
    fig = plt.gcf()
    fig.set_size_inches(18.5, 10.5)
    plt.show()
    fig.savefig('moves.png')
pie(moves)

#### Devide moves into:
    2* 46749 - DOWN
    46749 -    LEFT
    46749 -    RIGHT
    20749 -    ROT

In [ ]:
def delete(moves, grid, heights):
    lenght = moves[moves['move'] == 'DOWN'].shape[0]
    down = np.random.randint(0,lenght, 2*46749)
    _ = moves[moves['move'] == 'DOWN'].iloc[down]
    down = _.index
    
    lenght = moves[moves['move'] == 'RIGHT'].shape[0]
    right = np.random.randint(0,lenght, 46749)
    _ = moves[moves['move'] == 'RIGHT'].iloc[right]
    right = _.index
    
    lenght = moves[moves['move'] == 'ROT'].shape[0]
    rot = np.random.randint(0,lenght, 20749)
    _ = moves[moves['move'] == 'ROT'].iloc[rot]
    rot = _.index
    left = moves[moves['move'] == 'LEFT'].index
    
    indeces = np.hstack((down, right, rot, left))
    
    return moves.iloc[indeces], grid[indeces], heights[indeces]

In [ ]:
m, g,h = delete(moves, grids, heights)

In [ ]:
pie(m)

Mirror grids

In [ ]:
height_dist(np.hstack((h,np.fliplr(h))))

Add random height between 1,4 to last column

In [ ]:
g1 = g.copy()
h1 = h.copy()
def fill_column(grids, heights):
    for i, grid in enumerate(grids):
        if not i %2==0:
            height = np.random.randint(1,4)
            grid.T[9][19-height:20] =1 
            heights[i][9] = height
            
height_dist(h1)

In [ ]:
indeces = np.random.randint(0, h.shape[0],int(h.shape[0]/2))

In [ ]:
fill_column(g1, h1)

In [ ]:
height_dist(np.fliplr(h[h.T[0] != 0]))

In [ ]:
flipped =np.vstack((h1,np.fliplr(h[h.T[0] != 0]))).copy()
indeces = []
for i, element in enumerate(flipped.T[0]==0):
    if element:
        if i% 3==0:
            indeces.append(i)
    else:
        indeces.append(i)

In [ ]:
height_dist(np.vstack((h1,np.fliplr(h[h.T[0] != 0])))[np.array(indeces)])

In [ ]:
indeces1 = []
for i, element in enumerate(h.T[0] != 0):
    if element:
            indeces1.append(i)

In [ ]:
g = g[indeces1]

In [ ]:
for grid in g:
    np.fliplr(grid)
finalgrids = np.vstack((g1,g))[np.array(indeces)]

Flip moves:
    RIGHT -> LEFT
    LEFT -> RIGHT

In [ ]:
m = m.reset_index()
m1 = m.copy()
m = m.iloc[indeces1].reset_index()
m = m['move'].to_numpy()
for i, move in enumerate(m):
    if move == "RIGHT":
        m[i] = "LEFT"
    elif move == "LEFT":
        m[i] = "RIGHT"


In [ ]:
m = pd.DataFrame(m,columns = ["move"])


In [ ]:
m1 = m1.drop(['index'], axis=1)

In [ ]:
finalmoves = pd.concat([m1,m]).iloc[indeces]

In [ ]:
bad_grids = []
for i, grid in enumerate(finalgrids):
    if not np.any(grid):
        bad_grids.append(i)
non_zero = np.array(list(set(np.arange((finalmoves.shape[0]))) - set(bad_grids)))

In [ ]:
finalmoves = finalmoves.iloc[non_zero]
finalgrids = finalgrids[non_zero]

In [ ]:
finalmoves.drop('Unnamed: 0', axis = 1).to_csv('finalmoves.csv')

In [ ]:
with open("finalgrids.csv", "a") as outfile:
            for slice_2d in finalgrids:
                np.savetxt(outfile, slice_2d, fmt="%i", delimiter=",")

In [ ]:
heights = np.zeros((finalgrids.shape[0],10))
for i, grid in enumerate(finalgrids ):
    heights[i] = get_height(grid)
with open("finalheights.csv", "a") as outfile:
    np.savetxt(outfile,heights , fmt="%i", delimiter=",")

## Model 

In [ ]:
finalmoves = pd.read_csv('/kaggle/input/finaltetris/finalmoves.csv')
finalgrids = np.loadtxt('/kaggle/input/finaltetris/finalgrids.csv', delimiter=",")
finalgrids = finalgrids.reshape((int(finalgrids.shape[0]/20),20,10))

In [ ]:
finalheights =  np.loadtxt('/kaggle/input/heights/finalheights.csv', delimiter=",")
finalmoves = finalmoves.drop('Unnamed: 0', axis= 1)

Encode moves, shuffle and reshape grids 

In [ ]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]
def transform_g_m(g,y):
    
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    y = onehot_encoder.fit_transform(integer_encoded)
    X = g.reshape((-1,20,10,1))
    X = X.astype(np.short)
    y = y.astype(np.short)
    X, y = unison_shuffled_copies(X,y)
    return X,y

In [ ]:
X, y = transform_g_m(finalgrids,finalmoves)

In [ ]:

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Model
from tensorflow.keras.layers import concatenate


from tensorflow.keras.optimizers import schedules
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = SGD(lr_schedule)


input_ = Input(shape=(20,10,1))
x = Conv2D(1024, (2, 2) , activation='relu')(input_)
x = Conv2D(512, (2, 2) ,strides =(1,1), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(256, (1, 1) ,strides =(2,2), activation='tanh')(x)
x = Conv2D(128, (1, 1) ,strides =(1,1),padding='same', activation='tanh')(x)
x = Conv2D(64, (1, 1) ,strides =(1,1),padding='same', activation='tanh')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(128, activation='tanh')(x)
x = Dropout(0.1)(x)
x = Dense(64, activation='tanh')(x)
x= Dense(512, activation='tanh')(x)
x = Dropout(0.1)(x)
x = Dense(16, activation='tanh')(x)
x = Dense(4, activation='softmax')(x)
model = Model(inputs=input_, outputs=x)
model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
              metrics=['accuracy', "categorical_accuracy"])
model.summary()

In [ ]:
import keras
from tensorflow.keras.utils import plot_model
plot_model(model,show_shapes=True)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
             tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True), 
             keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.1, random_state=42, stratify = y)

In [ ]:
model.fit(x=X_train ,y=y_train,batch_size=64,epochs=30, validation_split=0.2, callbacks = callbacks)

### Model two inputs

In [ ]:
def unison_shuffled_copies(a, b, c):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p], c[p]
def transform_g_m(g,y,h):
    
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(y)
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    y = onehot_encoder.fit_transform(integer_encoded)
    X = g.reshape((-1,20,10,1))
    X = X.astype(np.short)
    y = y.astype(np.short)
    X, y,h = unison_shuffled_copies(X,y,h)
    return X,y,h

In [ ]:
X, Y , h = transform_g_m(finalgrids,finalmoves,finalheights)

In [ ]:

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Model
from tensorflow.keras.layers import concatenate


from tensorflow.keras.optimizers import schedules
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = SGD(lr_schedule)


inputA = Input(shape=(20,10,1))
x = Conv2D(512, (2, 2) , activation='relu')(inputA)
x = MaxPooling2D(strides =(1,1),padding='same')(x)
x = Conv2D(256, (2, 2) ,strides =(1,1), activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(512, (1, 1) ,strides =(2,2), activation='relu')(x)
x = Conv2D(128, (1, 1) ,strides =(1,1),padding='same', activation='relu')(x)
x = Conv2D(64, (1, 1) ,strides =(1,1),padding='same', activation='tanh')(x)
x = Flatten()(x)
x = Dense(512, activation='tanh')(x)
x = Dropout(0.2)(x)
x = Dense(64, activation='tanh')(x)
x = Model(inputs=inputA, outputs=x)

inputB = Input(shape=(10,))
y = Dense(128, activation="relu")(inputB)
y = Dropout(0.1)(y)
y = Dense(64, activation="relu")(y)
y = Dense(64, activation="tanh")(y)
y = Model(inputs=inputB, outputs=y)

combined = concatenate([x.output, y.output])


z= Dense(512, activation='tanh')(combined)
z = Dropout(0.1)(z)
z = Dense(16, activation='tanh')(z)
z = Dense(4, activation='softmax')(z)
model = Model(inputs=[x.input, y.input], outputs=z)



model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
              metrics=['accuracy', "categorical_accuracy"])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, h_train, h_test, y_train, y_test = train_test_split( X,h, Y, test_size=0.1, random_state=42, stratify = Y)

In [ ]:
import keras
from tensorflow.keras.utils import plot_model
plot_model(model,show_shapes=True)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
             tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True), keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)]



In [ ]:
model.fit(x=[X_train,h_train], y=y_train,batch_size=128,epochs=30, validation_split=0.2, callbacks = callbacks)

### AlexNet

In [ ]:

from tensorflow.keras.optimizers import SGD
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import schedules
from keras.models import Model
from tensorflow.keras.layers import concatenate


from tensorflow.keras.optimizers import schedules
lr_schedule = schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = SGD(lr_schedule)

#Input
inputA = Input(shape=(20,10,1))

# Layer 1
x = Conv2D(96, (2, 2) ,padding='same')(inputA)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Layer 2
x = Conv2D(256, (3, 3) ,strides =(1,1),padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 3
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(512, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# Layer 4
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(384, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# Layer 5
x = ZeroPadding2D((1, 1))(x)
x = Conv2D(1024, (3, 3), padding='same')(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)


# Layer 6
x = Flatten()(x)
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Dropout(0.5)(x)
# x = Model(inputs=inputA, outputs=x)

# inputB = Input(shape=(10,))

# y = Dense(128)(inputB)
# y = BatchNormalization()(y)
# y = Activation('relu')(y)
# y = Dropout(0.5)(y)

# y = Dense(512)(inputB)
# y = BatchNormalization()(y)
# y = Activation('relu')(y)
# y = Dropout(0.5)(y)
# y = Model(inputs=inputB, outputs=y)
# combined = concatenate([x.output, y.output])

z = Dense(512)(x)
z = BatchNormalization()(z)
z = Activation('relu')(z)
z = Dropout(0.5)(z)

# Layer 7
z = Dense(1024)(x)
z = BatchNormalization()(z)
z = Activation('relu')(z)
z = Dropout(0.5)(z)
z = Dense(4, activation='softmax')(z)

# model = Model(inputs=[x.input, y.input], outputs=z)
model = Model(inputs=inputA, outputs=z)


model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
              metrics=['accuracy', "categorical_accuracy"])
model.summary()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, h_train, h_test, y_train, y_test = train_test_split( X,h, Y, test_size=0.1, random_state=42, stratify = Y)
import keras
from tensorflow.keras.utils import plot_model
plot_model(model,show_shapes=True)
callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4),
             tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True), keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)]


In [ ]:
model.fit(x=X_train, y=y_train,batch_size=64,epochs=30, validation_split=0.2, callbacks = callbacks)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_val,y_pred))

In [ ]:
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
fpr, tpr, thresholds = roc_curve(y_val,y_pred)

plt.plot(fpr, tpr, label='ROC curve')
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.xlim([-0.02, 1])
plt.ylim([0, 1.02])
plt.legend(loc="lower right")